<a href="https://colab.research.google.com/github/seohyun126/LG-Aimers-6th/blob/main/Aimers%EB%AA%A8%EB%8D%B8%EB%A7%812%EC%B0%A8_IVF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/Aimers_data/train_IVF.csv')
test=pd.read_csv('/content/drive/MyDrive/Aimers_data/test_IVF.csv')

In [ ]:
train2=pd.read_csv('/content/drive/MyDrive/Aimers_data/train.csv')
test2=pd.read_csv("/content/drive/MyDrive/Aimers_data/test.csv")

In [ ]:
train2=train2[train2['특정 시술 유형']!='GIFT']
train2=train2[train2['특정 시술 유형']!='FER']
train2= train2.dropna(subset=['특정 시술 유형'])

In [ ]:
train2=train2[train2['시술 유형']=='IVF']

In [ ]:
test2=test2[test2['시술 유형']=='IVF']

In [ ]:
test2['ID']

,ID
0,TEST_00000
1,TEST_00001
2,TEST_00002
3,TEST_00003
4,TEST_00004
...,...
90062,TEST_90062
90063,TEST_90063
90064,TEST_90064
90065,TEST_90065


In [ ]:
print(train.shape)
print(test.shape)

(250054, 69)
(87891, 68)


In [ ]:
pd.DataFrame(train.isna().sum()[train.isna().sum() > 0])

,0
배아 이식 경과일,37271


In [ ]:
# 배아이식 경과일
num_cols=train.select_dtypes(exclude=['object']).columns
corr_values=train[num_cols].corr()['배아 이식 경과일']
corr_values[(corr_values >= 0.5) | (corr_values <= -0.5)]

,배아 이식 경과일
배란 자극 여부,0.586444
총 생성 배아 수,0.648159
수집된 신선 난자 수,0.575873
혼합된 난자 수,0.625844
파트너 정자와 혼합된 난자 수,0.564745
동결 배아 사용 여부,-0.690094
신선 배아 사용 여부,0.691271
배아 이식 경과일,1.000000


In [ ]:
# 상관관계가 높은 feature들로 채워줌
from sklearn.impute import KNNImputer

# 사용할 컬럼 선택
selected_columns = [
    '배란 자극 여부', '총 생성 배아 수', '수집된 신선 난자 수', '혼합된 난자 수',
    '파트너 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부','배아 이식 경과일'
]
# 선택한 컬럼만 포함한 데이터프레임 생성
train_selected = train[selected_columns]
# KNN Imputer 설정
imputer = KNNImputer(n_neighbors=5)  # K값은 5로 설정
train_selected_imputed = pd.DataFrame(imputer.fit_transform(train_selected), columns=selected_columns)

In [ ]:
# 원본 데이터프레임에 채워진 값 반영
train[selected_columns] = train_selected_imputed

In [ ]:
pd.DataFrame(train.isna().sum()[train.isna().sum() > 0])

,0


In [ ]:
# 선택한 컬럼만 포함한 데이터프레임 생성
test_selected = test[selected_columns]
# KNN Imputer 설정
imputer = KNNImputer(n_neighbors=3)  # K값은 3로 설정
test_selected_imputed = pd.DataFrame(imputer.fit_transform(test_selected), columns=selected_columns)
# 원본 데이터프레임에 채워진 값 반영
test[selected_columns] = test_selected_imputed

In [ ]:
pd.DataFrame(test.isna().sum()[test.isna().sum() > 0])

,0


In [ ]:
train.to_csv('/content/drive/MyDrive/Aimers_data/train_IVF.csv')
test.to_csv('/content/drive/MyDrive/Aimers_data/test_IVF.csv')

In [ ]:
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler, MinMaxScaler
numeric_columns = train.select_dtypes(include=['float64', 'int64']).columns  # 연속형 변수 선택
train_numeric = train[numeric_columns]

# 범주형 변수 제외 (0-1 사이 값은 범주형으로 간주)
train_numeric = train_numeric.loc[:, (train_numeric.min() != 0) | (train_numeric.max() != 1)]

# 2. Skewness 확인
skewness = train_numeric.apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("Skewness of numeric columns:\n", skewness)


Skewness of numeric columns:
 난자 혼합 경과일           33.586934
DI 출산 횟수            25.938452
DI 임신 횟수            22.351898
해동 난자 수             18.324040
기타 시술 횟수            16.325428
저장된 신선 난자 수         15.672372
PGS                 10.196136
PGD                  9.451222
DI 시술 횟수             6.861819
기증자 정자와 혼합된 난자 수     6.031707
미세주입 후 저장된 배아 수      5.332575
해동된 배아 수             5.126910
저장된 배아 수             3.776721
IVF 출산 횟수            2.557658
IVF 임신 횟수            2.389355
미세주입에서 생성된 배아 수      1.735090
미세주입된 난자 수           1.487226
IVF 시술 횟수            1.205798
총 생성 배아 수            1.186111
파트너 정자와 혼합된 난자 수     1.022407
혼합된 난자 수             0.994607
수집된 신선 난자 수          0.894733
미세주입 배아 이식 수         0.739645
보정된 나이               0.386741
이식된 배아 수            -0.275771
배아 이식 경과일           -0.424458
IUI                       NaN
Generic DI                NaN
IVI                       NaN
dtype: float64


In [ ]:
print(train['IUI'].value_counts())
print(train['Generic DI'].value_counts())
print(train['IVI'].value_counts())

IUI
0    250054
Name: count, dtype: int64
Generic DI
0    250054
Name: count, dtype: int64
IVI
0    250054
Name: count, dtype: int64


In [ ]:
train.drop(['IUI','Generic DI','IVI'],axis=1,inplace=True)
test.drop(['IUI','Generic DI','IVI'],axis=1,inplace=True)

In [ ]:
import numpy as np
train_numeric_log_transformed = train_numeric.copy()
for column in skewness.index:
    if skewness[column] > 1:  # 왜도가 1보다 큰 경우
        train_numeric_log_transformed[column] = train_numeric[column].apply(lambda x: np.log(x + 1))  # 로그 변환

In [ ]:
train_numeric_log_transformed.apply(lambda x: skew(x.dropna())).sort_values(ascending=False)

,0
난자 혼합 경과일,23.197797
DI 출산 횟수,21.074154
DI 임신 횟수,18.227508
해동 난자 수,13.939751
기타 시술 횟수,12.914974
저장된 신선 난자 수,10.707040
PGS,9.862352
PGD,9.106127
DI 시술 횟수,5.751723
기증자 정자와 혼합된 난자 수,4.082537


In [ ]:
train['PGD'].value_counts()

,count
PGD,
0.0,246858
2.0,2179
1.0,1017


In [ ]:
train['PGS'].value_counts()

,count
PGS,
0.0,247336
2.0,1929
1.0,789


In [ ]:
train.loc[train['난자 혼합 경과일']>1,'난자 혼합 경과일']=1
test.loc[test['난자 혼합 경과일']>1,'난자 혼합 경과일']=1

train.loc[train['DI 출산 횟수']>1,'DI 출산 횟수']=1
test.loc[test['DI 출산 횟수']>1,'DI 출산 횟수']=1

train.loc[train['DI 임신 횟수']>1,'DI 임신 횟수']=1
test.loc[test['DI 임신 횟수']>1,'DI 임신 횟수']=1


train.loc[train['해동 난자 수']>1,'해동 난자 수']=1
test.loc[test['해동 난자 수']>1,'해동 난자 수']=1


train.loc[train['기타 시술 횟수']>1,'기타 시술 횟수']=1
test.loc[test['기타 시술 횟수']>1,'기타 시술 횟수']=1

train.loc[train['저장된 신선 난자 수']>1,'저장된 신선 난자 수']=1
test.loc[test['저장된 신선 난자 수']>1,'저장된 신선 난자 수']=1

train.loc[train['PGS']>1,'PGS']=1
test.loc[test['PGS']>1,'PGS']=1

train.loc[train['PGD']>1,'PGD']=1
test.loc[test['PGD']>1,'PGD']=1

In [ ]:
train.select_dtypes(exclude=['float64', 'int64']).columns

Index([], dtype='object')

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 21.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from hyperopt import hp, fmin, tpe, Trials
from sklearn.metrics import roc_auc_score, make_scorer
#  데이터 준비
X = train.drop('임신 성공 여부',axis=1,inplace=False)
y = train['임신 성공 여부']  # 타겟 변수

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier

cat_features = X.loc[:, (X.min() == 0) & (X.max() == 1)].columns.tolist()
X[cat_features]=X[cat_features].astype(int)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = CatBoostClassifier(random_state=42, silent=True, task_type='CPU',cat_features=cat_features)

In [ ]:
param_dist = {
    'iterations': [100, 200],  # 트리의 개수 (간단한 범위로 축소)
    'depth': [2, 4, 6, 8, 10],  # 트리의 깊이
    'learning_rate': [0.01, 0.05, 0.005],  # 학습률
    'l2_leaf_reg': [1, 3],  # L2 규제 파라미터
}

# 3. StratifiedKFold로 불균형 데이터셋을 고려한 K-Fold 설정
cv = 3  # 교차 검증 횟수 줄이기

# 4. ROC-AUC를 평가 지표로 설정
roc_auc_scorer = make_scorer(roc_auc_score)

# 5. RandomizedSearchCV 실행
random_search = RandomizedSearchCV(
    estimator=model, param_distributions=param_dist,
    scoring=roc_auc_scorer, cv=cv, n_jobs=-1, verbose=2, n_iter=10  # 30개의 랜덤 조합 시도
)

# 모델 학습
random_search.fit(X, y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostClassifier object at 0x78156b0b5d10>,
                   n_jobs=-1,
                   param_distributions={'depth': [2, 4, 6, 8, 10],
                                        'iterations': [100, 200],
                                        'l2_leaf_reg': [1, 3],
                                        'learning_rate': [0.01, 0.05, 0.005]},
                   scoring=make_scorer(roc_auc_score, response_method='predict'),
                   verbose=2)

In [ ]:
# 최적 파라미터 출력
print("Best parameters found: ", random_search.best_params_)
# 최적 모델
best_model = random_search.best_estimator_

Best parameters found:  {'learning_rate': 0.05, 'l2_leaf_reg': 3, 'iterations': 200, 'depth': 4}


In [ ]:
test[cat_features]=test[cat_features].astype(int)
cat_test_preds = best_model.predict_proba(test)[:, 1]
test['임신 성공 여부'] = cat_test_preds

In [ ]:
feature_importance = best_model.get_feature_importance()

# feature importance를 DataFrame으로 변환
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importance
})

# feature importance 내림차순 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
feature_importance_df.head(20)

,Feature,Importance
27,이식된 배아 수,90.018059
44,보정된 나이,3.641402
43,배아 이식 경과일,2.033188
29,저장된 배아 수,1.323907
24,총 생성 배아 수,1.157333
22,IVF 출산 횟수,0.295729
20,IVF 임신 횟수,0.224270
0,배란 자극 여부,0.209004
18,IVF 시술 횟수,0.164219
35,혼합된 난자 수,0.158329


In [ ]:
test['ID']=test2['ID'].reset_index(drop=True)

In [ ]:
sub=pd.read_csv("/content/drive/MyDrive/Aimers_data/sample_submission3_2.csv",index_col=False)

In [ ]:
# 'test'와 'sub' 데이터를 ID를 기준으로 병합, sub 크기를 유지하도록 left join
merged = sub.merge(test[['ID', '임신 성공 여부']], on='ID', how='left')

# '임신 성공 여부' 값이 있는 경우에만 'prob'을 업데이트
merged['probability'] = merged['임신 성공 여부'].fillna(merged['probability'])

In [ ]:
merged.drop('임신 성공 여부',axis=1,inplace=True)

In [ ]:
merged[merged['probability']!=0].shape

(90067, 2)

In [ ]:
merged

,ID,probability
0,TEST_00000,0.002617
1,TEST_00001,0.001897
2,TEST_00002,0.155957
3,TEST_00003,0.112689
4,TEST_00004,0.477396
...,...,...
90062,TEST_90062,0.004328
90063,TEST_90063,0.271022
90064,TEST_90064,0.473028
90065,TEST_90065,0.152027


In [ ]:
merged[merged['probability']>0.5].shape[0]/merged.shape[0]

0.03579557440572018

In [ ]:
merged.to_csv("/content/drive/MyDrive/Aimers_data/sample_submission3_2.csv",index=False)

In [ ]:
merged

,ID,probability
0,TEST_00000,0.002617
1,TEST_00001,0.001897
2,TEST_00002,0.155957
3,TEST_00003,0.112689
4,TEST_00004,0.477396
...,...,...
90062,TEST_90062,0.004328
90063,TEST_90063,0.271022
90064,TEST_90064,0.473028
90065,TEST_90065,0.152027
